In [6]:
import pandas as pd
import requests

#Define API URL and paramters to fetch data
#url = "https://steamspy.com/api.php"
#params_top100in2weeks = {'request': 'top100in2weeks'}

#Define request variables and fetch data
#request_top100in2weeks = requests.get(url, params=params_top100in2weeks)

#Save data to files as pickles
#pd.DataFrame(request_top100in2weeks.json()).to_pickle("top100in2weeks.pkl")

#output = pd.read_pickle("top100in2weeks.pkl")
#print(output)

                                        570                               730  \
appid                                   570                               730   
name                                 Dota 2  Counter-Strike: Global Offensive   
developer                             Valve  Valve, Hidden Path Entertainment   
publisher                             Valve                             Valve   
score_rank                                                                      
positive                            1342423                           5292043   
negative                             259759                            712705   
userscore                                 0                                 0   
owners           100,000,000 .. 200,000,000         50,000,000 .. 100,000,000   
average_forever                       37741                             30884   
average_2weeks                         1498                               932   
median_forever              

In [1]:
import pandas as pd
import requests
import steamspypi

#44 pages as of March 29, 2021, try 45 (doesn't cause error)
all_steamspy_data = steamspypi.download_all_pages(num_pages=45)
#This creates json files with data




Sleeping for 70 seconds on Wed Nov 17 13:09:48 2021
Sleeping for 70 seconds on Wed Nov 17 13:10:58 2021
Sleeping for 70 seconds on Wed Nov 17 13:12:09 2021
Sleeping for 70 seconds on Wed Nov 17 13:13:19 2021
Sleeping for 70 seconds on Wed Nov 17 13:14:29 2021
Sleeping for 70 seconds on Wed Nov 17 13:15:39 2021
Sleeping for 70 seconds on Wed Nov 17 13:16:50 2021
Sleeping for 70 seconds on Wed Nov 17 13:18:00 2021
Sleeping for 70 seconds on Wed Nov 17 13:19:10 2021
Sleeping for 70 seconds on Wed Nov 17 13:20:20 2021
Sleeping for 70 seconds on Wed Nov 17 13:21:31 2021
Sleeping for 70 seconds on Wed Nov 17 13:22:41 2021
Sleeping for 70 seconds on Wed Nov 17 13:23:51 2021
Sleeping for 70 seconds on Wed Nov 17 13:25:02 2021
Sleeping for 70 seconds on Wed Nov 17 13:26:12 2021
Sleeping for 70 seconds on Wed Nov 17 13:27:22 2021
Sleeping for 70 seconds on Wed Nov 17 13:28:32 2021
Sleeping for 70 seconds on Wed Nov 17 13:29:43 2021
Sleeping for 70 seconds on Wed Nov 17 13:30:53 2021
Sleeping for

In [39]:
def get_list_filenames(filenamestart, fileending, num_files):
    filenames = []
    for i in range(num_files):
        filenames.append(filenamestart + str(i) + fileending)
        
    return filenames

import json
import pandas as pd               
import steamspypi
import time
def getAppDetails(appid):
    data_request = dict()
    data_request['request'] = 'appdetails'
    data_request['appid'] = appid
    data = steamspypi.download(data_request)
    time.sleep(1)
    return data

from pathlib import Path
def collect_details(appids, target_path):
    for appid in appids:
        filename = target_path + str(appid) + ".json"
        if not Path(filename).is_file():
            data = getAppDetails(appid)
            with open(filename, "w", encoding="utf8") as f:
                json.dump(data, f)
                
from glob import glob
def merge_details_from_json_folder(folder_path, target_file):
    #Merge details into one dataset
    details = dict()
    for file in glob(folder_path + "*.json"):
        with open(file, "r") as json_file:
            json_data = json.load(json_file)
            details[json_data['appid']] = json_data
            
    # Serializing json 
    json_object = json.dumps(details)
    
    if not Path(target_file).is_file():
        # Writing to sample.json
        with open(target_file, "w") as outfile:
            outfile.write(json_object)
    
    #Return dict so we can create a DataFrame for testing
    return details

import copy
def split_dict_to_multiple(input_dict, max_limit=200):
    #Splits dict into multiple dicts with given maximum size. 
    #Returns a list of dictionaries.
    chunks = []
    curr_dict ={}
    for k, v in input_dict.items():
        if len(curr_dict.keys()) < max_limit:
            curr_dict.update({k: v})
        else:
            chunks.append(copy.deepcopy(curr_dict))
            curr_dict = {k: v}
    # update last curr_dict
    chunks.append(curr_dict)
    return chunks

def combine_chunks_to_dict(chunks):
    new_dict = dict()
    for chunk in chunks:
        for key, value in chunk.items():
            new_dict[key] = value
            
    return new_dict

def save_dict_chunks(chunks, target_path):
    for index, chunk in enumerate(chunks):
        filename = target_path + str(index) + ".json"
        if not Path(filename).is_file():
            json_object = json.dumps(chunk)
            with open(filename, "w") as outfile:
                outfile.write(json_object)
        

In [22]:
#20211117_steamspy_page_0.json .. page_44.json
#Retrieve list of filenames containing steamspy basic data
filenames = get_list_filenames("20211117_steamspy_page_", ".json", 44)

In [23]:
import pandas as pd

#Collect appids from the 44 files
appids = []
for i in range(44):
    df = pd.read_json("20211117_steamspy_page_" + str(i) + ".json")
    for appid in list(df.loc["appid"]):
        appids.append(appid)

#Request details about each app via appid and save to specific json-files(appid.json)
collect_details(appids, "./app_details/")

In [24]:
from glob import glob
#Merge details into one dataset
details = merge_details_from_json_folder("./app_details/", "all_details.json")

In [32]:
#Minimize json file size by creating smaller chunks of details dict
chunks = split_dict_to_multiple(details, 1000)
save_dict_chunks(chunks, "./app_details_chunks/")

In [40]:
#Test remerging chunks
merged_details = combine_chunks_to_dict(chunks)

{10: {'appid': 10,
  'name': 'Counter-Strike',
  'developer': 'Valve',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 184180,
  'negative': 4787,
  'userscore': 0,
  'owners': '10,000,000 .. 20,000,000',
  'average_forever': 11054,
  'average_2weeks': 339,
  'median_forever': 220,
  'median_2weeks': 81,
  'price': '999',
  'initialprice': '999',
  'discount': '0',
  'ccu': 13899,
  'languages': 'English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean',
  'genre': 'Action',
  'tags': {'Action': 5370,
   'FPS': 4794,
   'Multiplayer': 3354,
   'Shooter': 3320,
   'Classic': 2750,
   'Team-Based': 1839,
   'First-Person': 1688,
   'Competitive': 1585,
   'Tactical': 1318,
   "1990's": 1177,
   'e-sports': 1170,
   'PvP': 861,
   'Old School': 744,
   'Military': 622,
   'Strategy': 607,
   'Survival': 294,
   'Score Attack': 285,
   '1980s': 256,
   'Assassin': 222,
   'Violent': 63}},
 1000000: {'appid': 1000000,
  'name': 'ASCENXION'

In [25]:
details_df = pd.DataFrame.from_dict(details, orient='index')

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
10,10,Counter-Strike,Valve,Valve,,184180,4787,0,"10,000,000 .. 20,000,000",11054,339,220,81,999,999,0,13899,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5370, 'FPS': 4794, 'Multiplayer': 3..."
1000000,1000000,ASCENXION,IndigoBlue Game Studio,PsychoFlux Entertainment,,26,4,0,"0 .. 20,000",0,0,0,0,999,999,0,0,"Korean, English, Simplified Chinese","Action, Adventure, Indie","{'Shoot 'Em Up': 186, 'Metroidvania': 181, 'Bu..."
1000010,1000010,Crown Trick,NEXT Studios,"Team17, NEXT Studios",,3568,526,0,"200,000 .. 500,000",489,0,615,0,799,1999,60,217,"Simplified Chinese, English, Japanese, Traditi...","Adventure, Indie, RPG, Strategy","{'Rogue-like': 260, 'Turn-Based Combat': 244, ..."
1000030,1000030,"Cook, Serve, Delicious! 3?!",Vertigo Gaming Inc.,Vertigo Gaming Inc.,,1347,79,0,"50,000 .. 100,000",310,0,367,0,1999,1999,0,41,English,"Action, Indie, Simulation, Strategy","{'Typing': 217, 'Casual': 208, 'Management': 2..."
1000040,1000040,细胞战争,DoubleC Games,DoubleC Games,,0,1,0,"0 .. 20,000",0,0,0,0,199,199,0,0,"English, Not supported, Simplified Chinese","Action, Casual, Indie, Simulation","{'Action': 22, 'Casual': 22, 'Indie': 21, 'Sim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999860,999860,Enemy On Board,Windwalk Games,Windwalk Games,,1145,402,0,"50,000 .. 100,000",82,0,82,0,0,0,0,4,English,"Action, Casual, Free to Play, Indie, Early Access","{'Free to Play': 240, 'Indie': 233, 'Multiplay..."
999880,999880,NASA's Exoplanet Excursions,"Caltech/IPAC, NASA's Jet Propulsion Laboratory","Caltech/IPAC, NASA's Jet Propulsion Laboratory",,11,1,0,"0 .. 20,000",0,0,0,0,0,0,0,0,English,Education,"{'Education': 23, 'VR': 6, 'Space': 5}"
999890,999890,Bruken,FlairBot Games,FlairBot Games,,1,0,0,"0 .. 20,000",0,0,0,0,299,299,0,0,English,"Adventure, Casual, Indie","{'Indie': 21, 'Casual': 21, 'Adventure': 20}"
999900,999900,Studio by RADiCAL,RADiCAL,"RADiCAL Solutions, LLC",,12,18,0,"20,000 .. 50,000",0,0,0,0,0,0,0,0,English,Animation & Modeling,"{'Animation & Modeling': 25, '3D': 13}"
